In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,499 kB]
Get:14 https://developer.dow

'/usr/local/lib/python3.10/dist-packages/pyspark'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("GraphFrameExample") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12") \
    .getOrCreate()

In [ ]:
#edges_df = spark.read.parquet("EdgesFinalExam/")
#edges_df.show()


In [ ]:
#vertics_df = spark.read.parquet("VertFinalExam/")
#vertics_df.show()

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

In [5]:
edges_schema = StructType([
    StructField("src", StringType(), True),
    StructField("dst", StringType(), True),
    StructField("tripid", IntegerType(), True),
    StructField("delay", IntegerType(), True),
    StructField("distance", IntegerType(), True)

])

In [6]:
vert_schema = StructType([
    StructField("id", StringType(), True),
    StructField("City", StringType(), True),
    StructField("State", StringType(), True),
    StructField("Country", StringType(), True),
])

In [7]:
# Streaming DataFrame for Edges
edges_df = spark.readStream \
    .schema(edges_schema) \
    .parquet("Edges_Data/")

In [8]:
# Streaming DataFrame for Edges
vertices_df = spark.readStream \
    .schema(vert_schema) \
    .parquet("Vertices_Data/")

In [9]:
edges_df.isStreaming

True

In [10]:
vertices_df.isStreaming

True

In [11]:
from pyspark.sql.functions import col, when

edges_stream_df = edges_df.withColumn(
    "delay_category",
    when(col("delay") < 0, "Early")
    .when(col("delay") > 0, "Late")
    .otherwise("OnTime")
)

In [12]:
vertices_stream_df = vertices_df.filter(col("state") != "")


In [13]:
edges_output_path = "Edges_Sink/"

edges_writer = edges_stream_df.writeStream \
    .format("parquet") \
    .option("path", edges_output_path) \
    .option("checkpointLocation", "Edges_Checkpoint/") \
    .outputMode("append")

In [14]:
vertices_output_path = "Vertices_Sink/"

vertices_writer = vertices_stream_df.writeStream \
    .format("parquet") \
    .option("path", vertices_output_path) \
    .option("checkpointLocation", "Vertices_Checkpoint/") \
    .outputMode("append")

In [18]:
q1 = edges_writer.start()


In [19]:
q1.stop()

In [20]:
q2 = vertices_writer.start()

In [21]:
q2.stop()

In [22]:
edg_df = spark.read.parquet("Edges_Sink/")
edg_df.show()

+---+---+-------+-----+--------+--------------+
|src|dst| tripid|delay|distance|delay_category|
+---+---+-------+-----+--------+--------------+
|RSW|EWR|1010630|  -10|     928|         Early|
|RSW|ORD|1021029|   87|     974|          Late|
|RSW|EWR|1021346|    0|     928|        OnTime|
|RSW|EWR|1021044|   18|     928|          Late|
|RSW|IAH|1021730|   29|     748|          Late|
|RSW|ORD|1020535|  605|     974|          Late|
|RSW|ORD|1021820|   71|     974|          Late|
|RSW|EWR|1021743|    0|     928|        OnTime|
|RSW|EWR|1022017|    0|     928|        OnTime|
|RSW|IAH|1020600|   -2|     748|         Early|
|RSW|CLE|1021214|   29|     891|          Late|
|RSW|EWR|1020630|   -5|     928|         Early|
|RSW|ORD|1031029|   13|     974|          Late|
|RSW|EWR|1031346|  279|     928|          Late|
|RSW|IAH|1031740|   29|     748|          Late|
|RSW|ORD|1030535|    0|     974|        OnTime|
|RSW|ORD|1031808|   -3|     974|         Early|
|RSW|DEN|1031516|   -2|    1396|        

In [23]:
vert_df = spark.read.parquet("Vertices_Sink/")
vert_df.show()

+---+-------------+-----+-------+
| id|         City|State|Country|
+---+-------------+-----+-------+
|ABE|    Allentown|   PA|    USA|
|ABI|      Abilene|   TX|    USA|
|ABQ|  Albuquerque|   NM|    USA|
|ABR|     Aberdeen|   SD|    USA|
|ABY|       Albany|   GA|    USA|
|ACK|    Nantucket|   MA|    USA|
|ACT|         Waco|   TX|    USA|
|ACV|       Eureka|   CA|    USA|
|ACY|Atlantic City|   NJ|    USA|
|ADQ|       Kodiak|   AK|    USA|
|AEX|   Alexandria|   LA|    USA|
|AGS|      Augusta|   GA|    USA|
|AHN|       Athens|   GA|    USA|
|AIA|     Alliance|   NE|    USA|
|AKN|  King Salmon|   AK|    USA|
|ALB|       Albany|   NY|    USA|
|ALO|     Waterloo|   IA|    USA|
|ALS|      Alamosa|   CO|    USA|
|ALW|  Walla Walla|   WA|    USA|
|AMA|     Amarillo|   TX|    USA|
+---+-------------+-----+-------+
only showing top 20 rows



In [ ]:
#print("Spark version:", spark.version)

In [24]:
!pip install graphframes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 4.4 MB/s eta 0:00:00


In [25]:
from graphframes import GraphFrame

In [26]:
g = GraphFrame(vert_df, edg_df)

# Run PageRank algorithm
results = g.pageRank(resetProbability=0.15, maxIter=5)

# Get top 10 vertices by rank
top_vertices = results.vertices.orderBy("pagerank", ascending=False).limit(10)

# Show the top vertices
top_vertices.show()


/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+---+--------------+-----+-------+------------------+
| id|          City|State|Country|          pagerank|
+---+--------------+-----+-------+------------------+
|ATL|       Atlanta|   GA|    USA|31.402169285067313|
|DFW|        Dallas|   TX|    USA| 22.76415219751248|
|ORD|       Chicago|   IL|    USA| 21.83241348762772|
|DEN|        Denver|   CO|    USA|16.026921025779515|
|LAX|   Los Angeles|   CA|    USA|14.358865452635795|
|IAH|       Houston|   TX|    USA|13.229634347806075|
|SFO| San Francisco|   CA|    USA|11.322517232690489|
|PHX|       Phoenix|   AZ|    USA|10.852423159730376|
|SLC|Salt Lake City|   UT|    USA| 9.622759351860472|
|LAS|     Las Vegas|   NV|    USA| 8.778471071473987|
+---+--------------+-----+-------+------------------+

